# Eliminación Gaussiana y Pivoteo Parcial Escalado

## Estabilidad Numérica, Estructura Algebraica y Evaluación del Error

## Resumen

Se desarrolla formalmente el método de eliminación Gaussiana como procedimiento directo para la resolución de sistemas lineales de la forma $ A x = b $. Se presenta su formulación matricial, su interpretación geométrica, su factorización implícita y el análisis del pivoteo parcial escalado como estrategia de control de propagación de errores de redondeo.


# 1. Planteamiento General

Considérese el sistema lineal:

$$
A x = b
$$

donde:

* $ A \in \mathbb{R}^{m \times n} $
* $ x \in \mathbb{R}^n $
* $ b \in \mathbb{R}^m $

En forma expandida:

$$
\begin{aligned}
a_{11}x_1 + a_{12}x_2 + \dots + a_{1n}x_n &= b_1 \\
a_{21}x_1 + a_{22}x_2 + \dots + a_{2n}x_n &= b_2 \\
\vdots \\
a_{m1}x_1 + a_{m2}x_2 + \dots + a_{mn}x_n &= b_m
\end{aligned}
$$

La matriz ampliada se define como:

$$
\tilde{A} = [A | b]
$$

# 2. Condiciones de Solución

Se verifica que el sistema admite solución si y solo si:

$$
\operatorname{rank}(A) = \operatorname{rank}([A|b])
$$

Para matrices cuadradas:

$$
A \text{ invertible } \iff \operatorname{rank}(A)=n \iff \ker(A)=\{0\}
$$

# 3. Eliminación Gaussiana

El método transforma ( A ) en una matriz triangular superior ( U ) mediante operaciones elementales por filas:

1. Intercambio de filas
2. Multiplicación por escalar no nulo
3. Suma de múltiplos de filas

El objetivo es obtener:

$$
Ux = b'
$$

donde:

$$
U =
\begin{pmatrix}
u_{11} & u_{12} & \dots & u_{1n} \\
0 & u_{22} & \dots & u_{2n} \\
\vdots \\
0 & 0 & \dots & u_{nn}
\end{pmatrix}
$$

Posteriormente se aplica sustitución regresiva:

$$
x_k = \frac{1}{u_{kk}} \left( b'_k - \sum_{j=k+1}^{n} u_{kj} x_j \right)$$

# 4. Problema Numérico: Errores de Redondeo

En aritmética de precisión finita, si:

$$
|a_{qq}| \ll |a_{qj}|
$$

la división por un pivote pequeño amplifica errores.

Ejemplo clásico:

$$
0.0002x_1 + 1.471x_2 = 1.473
$$
$$
0.2346x_1 - 1.317x_2 = 1.029
$$

La eliminación directa genera error relativo significativo en $x_1$.

# 5. Pivoteo Parcial Escalado

En cada columna $q$:

1. Se selecciona el pivote $a_{kq}$ tal que:

$$
|a_{kq}| = \max_{i \ge q} |a_{iq}|
$$

2. Se intercambian filas si es necesario.
3. Se procede con eliminación.

Este criterio reduce el crecimiento del error.

# 6. Implementación Computacional

## 6.1 Eliminación Gaussiana sin Pivoteo

In [6]:
import numpy as np

def gauss_elimination(A, b):
    A = A.astype(float)
    b = b.astype(float)
    n = len(b)

    # Eliminación
    for k in range(n-1):
        for i in range(k+1, n):
            m = A[i,k] / A[k,k]
            A[i,k:] -= m * A[k,k:]
            b[i] -= m * b[k]

    # Sustitución regresiva
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (b[i] - np.dot(A[i,i+1:], x[i+1:])) / A[i,i]

    return x

## 6.2 Eliminación con Pivoteo Parcial

In [2]:
def gauss_partial_pivot(A, b):
    A = A.astype(float)
    b = b.astype(float)
    n = len(b)

    for k in range(n-1):
        # Selección del pivote
        max_row = np.argmax(abs(A[k:,k])) + k
        if A[max_row,k] == 0:
            raise ValueError("Matriz singular")
        # Intercambio
        A[[k,max_row]] = A[[max_row,k]]
        b[[k,max_row]] = b[[max_row,k]]

        for i in range(k+1, n):
            m = A[i,k] / A[k,k]
            A[i,k:] -= m * A[k,k:]
            b[i] -= m * b[k]

    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (b[i] - np.dot(A[i,i+1:], x[i+1:])) / A[i,i]

    return x

# 7. Vector Residual y Error Relativo

Definiciones:

$$
r = b - Ax
$$

$$
\text{Error relativo} = \frac{|x - x_{exacto}|}{|x_{exacto}|}
$$

Implementación:

In [7]:
def residual(A, x, b):
    return b - A @ x

def relative_error(x, x_exact):
    return np.linalg.norm(x - x_exact) / np.linalg.norm(x_exact)

# 8. Ejemplo Numérico

In [8]:
A = np.array([[0.0002, 1.471],
              [0.2346, -1.317]])

b = np.array([1.473, 1.029])

x_no_pivot = gauss_elimination(A.copy(), b.copy())
x_pivot = gauss_partial_pivot(A.copy(), b.copy())

print("Sin pivoteo:", x_no_pivot)
print("Con pivoteo:", x_pivot)
print("Residual:", residual(A, x_pivot, b))

Sin pivoteo: [10.  1.]
Con pivoteo: [10.  1.]
Residual: [ 0.00000000e+00 -2.22044605e-16]


# 9. Interpretación Geométrica

En dimensión 2:

* Cada ecuación representa una recta.
* La solución es el punto de intersección.
* Si $ \det(A)=0 $, las rectas son paralelas o coincidentes.

# 10. Complejidad Computacional

El algoritmo requiere aproximadamente:

$$
\frac{2}{3}n^3
$$

operaciones aritméticas.

# 11. Conclusiones

La eliminación Gaussiana constituye:

* Un método directo finito.
* Una factorización implícita $A = LU$.
* Un procedimiento sensible a la magnitud del pivote.
* Un algoritmo cuya estabilidad mejora mediante pivoteo parcial escalado.

El análisis del residual permite evaluar estabilidad numérica.
El pivoteo parcial reduce crecimiento del error y mantiene estabilidad estructural.

En sistemas de gran escala, constituye la base de métodos más avanzados como factorizaciones LU con pivotaje, descomposición QR y métodos iterativos precondicionados.
